In [13]:
import pandas as pd
import sklearn as skl
from sklearn import *
%matplotlib inline
import matplotlib.pyplot as plt
import xgboost as xgb
import numpy as np

In [96]:
pd.set_option('max_columns', 30)
fulldata = pd.read_csv('data.csv');
fulldata.head(4)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,0,2000-01,27,18,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,0,2000-01,22,15,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,0,2000-01,45,16,1.0,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,0,2000-01,52,22,0.0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4


In [97]:
text_columns = ['action_type', 'season', 'game_date', 'combined_shot_type', 'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'team_name', 'matchup', 'opponent']
for col in text_columns:
    uniq_words = fulldata[col].unique()
    uniq_words = dict(enumerate(uniq_words, 1)) # 1 to enumerate from 1
    uniq_words = {v:k for k,v in uniq_words.items()}
    fulldata = fulldata.replace({col: uniq_words})
fulldata.head(4)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,1,1,10,20000012,33.9723,167,72,-118.1028,10,1,0,1,27,18,NaN,1,1,1,1,1610612747,1,1,1,1,1
1,1,1,12,20000012,34.0443,-157,0,-118.4268,10,1,0,1,22,15,0.0,1,2,1,2,1610612747,1,1,1,1,2
2,1,1,35,20000012,33.9093,-101,135,-118.3708,7,1,0,1,45,16,1.0,1,3,1,1,1610612747,1,1,1,1,3
3,1,1,43,20000012,33.8693,138,175,-118.1318,6,1,0,1,52,22,0.0,1,4,1,1,1610612747,1,1,1,1,4


In [100]:
data_test = fulldata[pd.isnull(fulldata['shot_made_flag'])]
data = fulldata[pd.notnull(fulldata['shot_made_flag'])]
data.head(4)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,playoffs,season,seconds_remaining,shot_distance,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
1,1,1,12,20000012,34.0443,-157,0,-118.4268,10,1,0,1,22,15,0.0,1,2,1,2,1610612747,1,1,1,1,2
2,1,1,35,20000012,33.9093,-101,135,-118.3708,7,1,0,1,45,16,1.0,1,3,1,1,1610612747,1,1,1,1,3
3,1,1,43,20000012,33.8693,138,175,-118.1318,6,1,0,1,52,22,0.0,1,4,1,1,1610612747,1,1,1,1,4
4,2,2,155,20000012,34.0443,0,0,-118.2698,6,2,0,1,19,0,1.0,1,5,2,3,1610612747,1,1,1,1,5


In [88]:
dTrain, dTest, YTrain, YTest = model_selection.train_test_split(data.ix[:, data.columns!='shot_made_flag'], data['shot_made_flag'], test_size=0.3, random_state=1)

In [102]:
XGBoost = xgb.XGBClassifier()
xgbRegr = xgb.XGBRegressor()

In [161]:
XGBoost.fit(dTrain, YTrain)
r_X, r_Y = data.ix[:, data.columns!='shot_made_flag'],  data['shot_made_flag']
xgbRegr.fit(r_X, r_Y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [104]:
aucXGBTrain = metrics.roc_auc_score(YTrain, XGBoost.predict_proba(dTrain)[:,1])
aucXGBTest = metrics.roc_auc_score(YTest, XGBoost.predict_proba(dTest)[:,1])
print('XGBoost:       {:6.4}  {:6.4}'.format(aucXGBTrain, aucXGBTest))

XGBoost:       0.7338  0.7009


In [153]:
indices = [x+1 for x in data_test.index]

In [154]:
test_acc = XGBoost.predict(data_test.ix[:, data_test.columns!='shot_made_flag'])
res = list(zip(indices, test_acc))
for z in res[:3]:
    print(z)

(1, 0.0)
(8, 1.0)
(17, 1.0)


In [157]:
r_test_acc = xgbRegr.predict(data_test.ix[:, data_test.columns!='shot_made_flag'])
r_res = list(zip(indices, r_test_acc))
for z in r_res[:3]:
    print(z)

(1, 0.46669531)
(8, 0.65671176)
(17, 0.77790439)


In [159]:
with open('result_classiffier.csv', 'w') as f:
    f.write('shot_id,shot_made_flag\n')
    for s,m in res:
        f.write(str(s) + ',' +str(m)+'\n')

In [158]:
with open('result_regressor.csv', 'w') as f:
    f.write('shot_id,shot_made_flag\n')
    for s,m in r_res:
        f.write(str(s) + ',' +str(m)+'\n')

In [162]:
xgbRegr.fit(r_X, r_Y)
r_test_acc = xgbRegr.predict(data_test.ix[:, data_test.columns!='shot_made_flag'])
r_res = list(zip(indices, r_test_acc))
for z in r_res[:3]:
    print(z)

(1, 0.28595519)
(8, 0.45807004)
(17, 0.70314646)
